<a href="https://colab.research.google.com/github/Chuck2Win/HAAFOR/blob/master/Haafor_%EC%9D%98%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from pandas import DataFrame as df
from matplotlib import pyplot as plt
from google.colab import drive
import os 
import re
import gzip
import csv
drive.mount('/content/gdrive')
new_path=('/content/gdrive/My Drive/project')
os.chdir(new_path)
print(os.getcwd())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/project


In [ ]:
! pip install bpemb

     |████████████████████████████████| 1.1MB 4.2MB/s 


In [ ]:
from bpemb import BPEmb 
bpemb_en=BPEmb(lang='en',dim=100, vs=5000) #100차원으로, 10000개의 단어

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

In [ ]:
config=Config({'n_layers':2,'n_head':2,'d_model':100,'hidden_dim':4*100,'padding_idx':1,'seq_len':500,'batch_size':32,'dropout':0.3,'max_len':5000})

In [ ]:
# bpemb_en word2vec 저장
bpemb_en.emb.save_word2vec_format('./pretrained/wv_5000')
# torchtext로 구성해낸다.
# 관건 align 작업
vectors=torchtext.vocab.Vectors('wv_5000',cache='./pretrained') #'./wv'모델을 읽어온다.
# 안될 때엔 변수 싹 다 지우고 다시 해라...-> drive에서

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
 99%|█████████▉| 4942/5000 [00:00<00:00, 15584.71it/s]


In [ ]:
# 데이터 분석하기 전에 torchtext를 활용하기
# torchtext에 있는 사전 훈련된 임베딩 벡터 활용하기
# torchtext iterator는 drop_last를 못한다.
import torchtext
TEXT=torchtext.data.Field(sequential=True,use_vocab=True,init_token='<s>',eos_token='</s>',tokenize=bpemb_en.encode,fix_length=config.seq_len,batch_first=True,lower=True,pad_token='<pad>',unk_token='<unk>') 
ISNEXT=torchtext.data.Field(sequential=False,use_vocab=False,batch_first=True,is_target=True) 
Train_data=torchtext.data.TabularDataset('./train_data_batch_swap.csv',format='csv',fields=[('A',TEXT),('B',TEXT),('NEXT',ISNEXT)],skip_header=True) # 이 때 train data는 sentence형태 이여야함.(tokenized가 되지 않은 상태)
TEXT.build_vocab(Train_data,vectors=vectors)
print(len(TEXT.vocab.stoi))
config['n_token']=len(TEXT.vocab.stoi)

4661


In [ ]:
# test data
Test_data=torchtext.data.TabularDataset('./test_data_swap.csv',format='csv',fields=[('A',TEXT),('B',TEXT),('NEXT',ISNEXT)],skip_header=True)
# test loader
test_loader=torchtext.data.Iterator(Test_data,batch_size=100) 

In [ ]:
# val data
val_data=torchtext.data.TabularDataset('./val_data_swap.csv',format='csv',fields=[('A',TEXT),('B',TEXT),('NEXT',ISNEXT)],skip_header=True)
# val loader
val_loader=torchtext.data.Iterator(val_data,batch_size=100)

In [ ]:
# train loader
train_loader=torchtext.data.Iterator(Train_data,batch_size=config.batch_size) 

In [ ]:
# model - attention model을 활용한다.
import math
# positional encoding

class PositionalEncoding(nn.Module):

    def __init__(self, config):
        super(PositionalEncoding, self).__init__()
        self.config=config
        self.dropout = nn.Dropout(p=self.config.dropout)
        self.pe = torch.zeros(config.max_len, config.d_model)
        position = torch.arange(0, config.max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, config.d_model, 2).float() * (-math.log(10000.0) / config.d_model))
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = self.pe.unsqueeze(0).transpose(0,1) # max len, 1, d_model
        

    def forward(self, x):
        '''
        x shape : seq len, batch size, d model
        '''
        self.pe=self.pe.to(x.device)
        x = x + self.pe[:x.size(0),:,:] # 후 항 shape : seq len, 1, d model
        return self.dropout(x)

In [ ]:
class TransformerModel(nn.Module):

    def __init__(self, config,vectors):
        super(TransformerModel, self).__init__()
        self.config=config
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        self.pos_encoder = PositionalEncoding(self.config)
        encoder_layers = TransformerEncoderLayer(self.config.d_model,self.config.n_head, self.config.hidden_dim, self.config.dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, self.config.n_layers)
        decoder_layers = TransformerDecoderLayer(self.config.d_model,self.config.n_head, self.config.hidden_dim, self.config.dropout)
        self.transformer_decoder = TransformerDecoder(decoder_layers, self.config.n_layers)
        self.embedding = nn.Embedding.from_pretrained(vectors,freeze=True, padding_idx=self.config.padding_idx)
        self.fc=nn.Sequential(nn.Linear(self.config.d_model*self.config.seq_len,self.config.seq_len),nn.BatchNorm1d(self.config.seq_len),nn.ReLU(),nn.Linear(self.config.seq_len,1),nn.Sigmoid())
        
    def gen_padding_mask(self, input):
        '''
        input shape : seq len, batch size
        embedding(input) : seq len, batch size, d_model
        mask shape : seq len, batch size <- seq len에서 padding idx인 녀석은 1, 나머지는 0
        근데 TransformerEncoder에 넣어주기 위해선 batch size, seq len으로 바꿔줘야 한다.
        subsquent mask와는 다르다. -> TransformerEncoder에선 src_key_padding_mask의 input으로 들어감
        '''
        
        mask=input.eq(self.config.padding_idx).T 
        return mask
    
    def forward(self, src,tgt):
        device=src.device
        src=src.T
        tgt=tgt.T
        src_key_padding_mask = self.gen_padding_mask(src)
        tgt_key_padding_mask = self.gen_padding_mask(tgt)
        self.src_key_padding_mask = src_key_padding_mask.to(device)
        self.tgt_key_padding_mask = tgt_key_padding_mask.to(device)
        src_out = self.embedding(src) * math.sqrt(self.config.d_model)
        src_out = self.pos_encoder(src_out)
        src_out = self.transformer_encoder(src_out,src_key_padding_mask=self.src_key_padding_mask)
        tgt_out = self.embedding(tgt) * math.sqrt(self.config.d_model)
        tgt_out = self.pos_encoder(tgt_out)
        out1 = self.transformer_decoder(tgt_out,src_out,tgt_key_padding_mask=self.tgt_key_padding_mask,
                                         memory_key_padding_mask=self.src_key_padding_mask)
        out1 = out1.transpose(0,1) # batch size, seq_len, d_model
        out2 = out1.reshape(-1,self.config.seq_len*self.config.d_model)
        output = self.fc(out2) #  batch size, 1
        return output.reshape(-1)

In [ ]:
os.chdir('./experiment의식')

In [ ]:
def train():
    model.train() # 학습 모드를 시작합니다.
    for _,ok in enumerate(train_loader):
        src,tgt,is_next=ok.A,ok.B,ok.NEXT
        src=src.to(device)
        tgt=tgt.to(device)
        is_next=is_next.float().to(device)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # clip을 해준다., 다음 번에는 안 해줘야겠다.
        output = model(src,tgt)
        loss = criterion(output, is_next)
        loss.backward()
        acc = ((output>0.5).float() == is_next).float().mean()
        total_acc += acc.item() 
        optimizer.step()
        n+=1
        total_loss += loss.item()   

In [ ]:
len(optimizer.state_dict()['param_groups']

1

[4.75]

In [ ]:
# train
import time
device = 'cpu'
model = TransformerModel(config,TEXT.vocab.vectors).to(device)
criterion = nn.BCELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

epochs = 20
start_time = time.time()
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    print(scheduler.get_last_lr()[0])
    train()
    scheduler.step()
    print(optimizer.state_dict()['param_groups'][0]['lr'])                                                                                                                                                                               
    with torch.no_grad():
        model.eval()
        total_loss=0.
        total_acc=0.
        n=0
        for i in train_loader:
            src,tgt,is_next=i.A,i.B,i.NEXT
            src=src.to(device)
            tgt=tgt.to(device)
            is_next=is_next.float().to(device)
            output = model(src,tgt)
            loss = criterion(output, is_next)
            acc = ((output>0.5).float() == is_next).float().mean()
            total_acc += acc.item() 
            n+=1
            total_loss += loss.item()
        print('-' * 89)
        print('| epoch %d | train_loss %.2f | train_acc %.2f | processed_time %.1f'%(
                epoch, total_loss/n, total_acc/n, time.time()-epoch_start_time))      
        print('-' * 89)
        val_loss=0.
        val_acc=0.
        val_n=0
        for i in val_loader:
            src,tgt,is_next=i.A,i.B,i.NEXT
            src=src.to(device)
            tgt=tgt.to(device)
            is_next=is_next.float().to(device)
            output = model(src,tgt)
            loss = criterion(output, is_next)
            acc = ((output>0.5).float() == is_next).float().mean()
            val_acc += acc.item() 
            val_n+=1
            val_loss += loss.item()
        print('-' * 89)
            
        print('| epoch %d | val_loss %.2f | val_acc %.2f | processed_time %.1f'%(
                epoch, val_loss/val_n, val_acc/val_n, time.time()-epoch_start_time)) 
        print('-' * 89)


0.005
0.00475
-----------------------------------------------------------------------------------------
| epoch 1 | train_loss 28.57 | train_acc 0.51 | processed_time 111.2
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| epoch 1 | val_loss 92.99 | val_acc 0.51 | processed_time 117.3
-----------------------------------------------------------------------------------------
0.00475
0.0045125
-----------------------------------------------------------------------------------------
| epoch 2 | train_loss 32.94 | train_acc 0.50 | processed_time 110.7
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| epoch 2 | val_loss 104.61 | val_acc 0.49 | processed_time 116.9
---------------------------------------------------------------------